In [13]:
!pip install keras-tuner

In [14]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [15]:
fashion_mnist=keras.datasets.fashion_mnist

In [16]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [17]:
train_images=train_images/225.0
test_images=test_images/225.0

In [18]:
train_images[0].shape

(28, 28)

In [19]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [24]:
def build_model(hp):
  model=keras.Sequential([
      keras.layers.Conv2D(
      filters=hp.Int('conv_1_filer',min_value=32,max_value=128,step=16),
      kernel_size=hp.Choice('conv_1_kernel',values= [3,5]),
      activation='relu',
      input_shape=(28,28,1)
  ),


keras.layers.Conv2D(
      filters=hp.Int('conv_2_filer',min_value=32,max_value=64,step=16),
      kernel_size=hp.Choice('conv_2_kernel',values= [3,5]),
      activation='relu'
    
  ),

keras.layers.Flatten(),
keras.layers.Dense(
    units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
    activation='relu'
),
keras.layers.Dense(10,activation='softmax')#output layer
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model
                    

In [25]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [27]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                      max_trials=5,directory='output',project_name="Mnist fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)


Search: Running Trial #7

Hyperparameter    |Value             |Best Value So Far 
conv_1_filer      |64                |48                
conv_1_kernel     |5                 |3                 
conv_2_filer      |32                |48                
conv_2_kernel     |5                 |3                 
dense_1_units     |128               |48                
learning_rate     |0.01              |0.001             

Epoch 1/3
 583/1688 [=========>....................] - ETA: 3:20 - loss: 0.7182 - accuracy: 0.7728

In [32]:
model=tuner_search.get_best_models(num_models=1)[0]

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        20784     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1327152   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,348,906
Trainable params: 1,348,906
Non-trainable params: 0
_________________________________________________________________
